## 복습: Chain의 기본 구조 = 프롬프트 + 모델 + 출력 파서

In [ ]:
from dotenv import load_dotenv
# API KEY 정보로드
load_dotenv()

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# output_parser 객체 생성
output_parser = StrOutputParser()

# prompt_template 객체 생성
template = "{country}의 수도를 알려줘"
prompt_template = PromptTemplate.from_template(template)

# llm 모델 객체 생성
llm = ChatOpenAI(model_name='gpt-4o-mini')

input = {
    'country': '대한민국'
}

# chain에 output_parser 추가
chain = prompt_template | llm | output_parser

chain.invoke(input)

## 프롬프트 템플릿에 복수 input_variables를 추가하려면?

prompt_template의 input_variables는 리스트로 관리되며, 실제 변수 값은 딕셔너리 형태로 전달하여 주입할 수 있음.

In [ ]:
# prompt_template에 두 개의 input_variables 설정
template = "{country}의 {interests}를 알려줘"
prompt_template = PromptTemplate.from_template(template)

prompt_template

In [ ]:
# input_variables의 데이터를 딕셔너리로 셋팅
input = {
    'country': '대한민국',
    'interests': '인구'
}

llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.1)

chain = prompt_template | llm | output_parser

# chain에 input_variables 전달 후 invoke
chain.invoke(input)

※ 실습: 나만의 여행 가이드 프롬프트 만들기

아래와 같은 문장을 생성하는 PromptTemplate을 작성하고 실행해보시오.

+ {city}, {activity} 두 개의 input_variables를 사용하는 템플릿을 만들 것

+ "{도시명}에서 즐길 수 있는 {활동 종류}를 추천해줘."

    - 예시 문장:

      "부산에서 즐길 수 있는 해양 스포츠를 추천해줘."

In [ ]:
# YOUR CODE

## 워크플로우 설계 및 구현

사용자에게 어떠한 응답을 제공하는 domain 특화 Large-Language-Model(LLM)을 개발하고 싶나요?

선생님의 논문에서 개발한 시스템 중, Code Review Module의 워크플로우를 살펴봅시다.

### What is a workflow?

- A workflow is a system for managing repetitive processes and tasks which occur in a particular order.
- LangChain을 활용하면 이러한 워크플로우를 모듈 단위로 분리하고, 
  
  재사용 가능하게 구성할 수 있어 LLM 기반 시스템의 구조적 설계와 유지보수가 쉬워집니다.

### What is a domain 특화 LLM?

- 의료, 법률, 교육, 소프트웨어 등 특정 분야의 지식과 언어 패턴에 맞춰 튜닝된 LLM을 말합니다.
- 일반적인 LLM은 폭넓은 주제를 다룰 수 있지만, 

  도메인 특화 LLM은 특정 분야의 정확하고 신뢰할 수 있는 응답을 생성하는 데 강점을 가집니다.

### 분기 기반 순차 실행으로 정교한 도메인 LLM 만들기

다음 링크를 통해, domain 특화 LLM의 한 예시인 선생님이 구현한 **CRM 워크플로우의 구조**를 살펴봅시다.

- 링크: https://github.com/dkgkejdrb/CodeTutorBot

In [ ]:
# CRM 중, Review Necessity Chain
from dotenv import load_dotenv
# API KEY 정보로드
load_dotenv()

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

RNC_prompt_template = PromptTemplate.from_template(
"""
    Before reviewing the [SubmittedCode], determine whether a review is necessary.
                
    - Respond 'yes' if the [SubmittedCode] contains mistakes or improvements are needed.
    - Respond 'no_correct' if the [SubmittedCode] is already correct and matches the [Solution], meaning no review is needed.
    - Respond 'no_meaningless' if the [SubmittedCode] is too simple (e.g., 'print()', single numbers, random characters) and does not attempt to solve the [PythonProblem], meaning no review is needed.
        
    Respond with only one of the three options: 'yes', 'no_correct', or 'no_meaningless'.
        
    PythonProblem: {pythonProblem}
    Submitted Code: {submittedCode}
    Solution: {solution}
"""
)

RNC_llm = ChatOpenAI( 
    model_name = "gpt-4o",
    temperature = 0.2,
    max_tokens = 50,
    top_p = 0.1
)

RNC_input = {
    'pythonProblem': """
        [문제]
        print() 함수와 sep, end 옵션을 이용하여 "출력 예시"와 같이 출력해 봅시다.

        [테스트]
        입력 예시: 
        없음

        출력 예시:
        '코로나 블루', 극복하기!
    """,
    'submittedCode': """
        print("'코로나 블루'","극복하기")
    """,
    'solution': """
        print("'코로나 블루'","극복하기",sep=', ',end=! )
    """
}

RNC = RNC_prompt_template | RNC_llm | output_parser
RNC_response = RNC.invoke(RNC_input)
print(RNC_response)

In [ ]:
# 중요★) 분기RNC_response = yes 인 경우에만 Review Comment Generation Chain 실행
if not RNC_response == 'yes': print('코드 리뷰 필요 없음')
else:
    # CRM 중, Review Comment Generation Chain
    RCGC_styleTone = """
        Review with vocabulary difficulty level that primary and secondary school students can understand.
    """
    RCGC_instruction = """
        As in the example shown in [Example], [RC][/RC] and [R][/R] must be included in the response. 
        In [SubmittedCode], add ‘Code to fix’ comment to the end of incorrect code lines.
        Reply to [RC][/RC] with commented [SubmittedCode] as is.
        Respond to code reviews with [R][/R] in a ‘polite tone’ and within three sentences and emoji.
    """
    RCGC_restriction = """
        Must never present the fixed code and [Solution] to [RC][/RC] and [R][/R].
        Do not include code fences such as \`\`\`python or any similar syntax in your response.
    """
    RCGC_example ="""

    """
    
    RCGC_prompt_template = PromptTemplate.from_template(
        RCGC_styleTone + RCGC_instruction + RCGC_restriction +
    """
        PythonProblem: {pythonProblem}
        SubmittedCode: {submittedCode}
        Solution: {solution}
        Example: {example}
    """
    )

    RCGC_llm = ChatOpenAI( 
        model_name = "gpt-4o",
        temperature = 0.2,
        max_tokens = 320,
        top_p = 0.8
    )

    RCGC_input = {
        'pythonProblem': """
            [문제]
            print() 함수와 sep, end 옵션을 이용하여 "출력 예시"와 같이 출력해 봅시다.

            [테스트]
            입력 예시: 
            없음

            출력 예시:
            '코로나 블루', 극복하기!
        """,
        'submittedCode': """
            print("'코로나 블루'","극복하기")
        """,
        'solution': """
            print("'코로나 블루'","극복하기",sep=', ',end=! )
        """,
        'example': """
            [RC]
            length = 42.195
            print("Marathon’s distence %.fkm" %length) # Code to fix
            print("Marathon’s distance is %.2fkm" %length)
            [/RC]

            [R]
            There is a typo in the print function in the first line. The correct expression is ‘distance’ instead of ‘distence’. Check for typos with a little more attention!
            [/R]
        """
    }

    RCGC = RCGC_prompt_template | RCGC_llm | output_parser
    RCGC_response = RCGC.invoke(RCGC_input)
    print(RCGC_response)

※ 실습: 감정 판단 체인 만들기

+ 감정 판단 체인 →
	- text가 부정적인 감정이라면 '예', 아니면 '아니오'라고 답변하는 체인


+ 분기 처리 →
	- 감정 판단 체인의 응답 결과가 "예"일 경우: 다음 체인 실행
	- 감정 판단 체인의 응답 결과가 "아니오"일 경우: "리뷰가 필요하지 않습니다." 출력

+ 리뷰 생성 체인 →
	- text에서 어떤 부분이 부정적인지 친절하게 알려주는 체인

+ {"text": "나는 오늘 너무 피곤하고 지쳤어."}를 입력하여 체인을 실행할 것

In [ ]:
# YOUR CODE

## Chain을 병렬로 실행할 수 있을까?

RunnableParallel을 사용하여, 복수의 chain을 동시에 실행할 수 있다.

### Runnable 이란?
Runnable은 LangChain에서 입력을 받아 출력을 생성하는 모든 실행 단위를 의미함.

LLM, PromptTemplate, Parser 등 대부분의 LangChain 구성 요소는 Runnable이라고 함.

즉, .invoke(input)으로 실행할 수 있는 객체는 전부 Runnable이라고 보면 됨.

In [ ]:
# Chain 을 구성하는 요소는 전부 Runnable이라 볼 수 있음
prompt = PromptTemplate.from_template("Translate in Korean: {text}")
# prompt.invoke({'text': "hello?"})
llm = ChatOpenAI()
chain = prompt | llm  # 둘 다 Runnable이기 때문에 체이닝 가능
# chain.invoke({'text': "hello?"})

### RunnableParallel을 사용한 chain 병렬 실행 예시 #1

- 체인 1: 나라의 수도를 생성
- 체인 2: 나라의 인구를 생성

In [ ]:
from langchain_core.runnables import RunnableParallel

chain1 = PromptTemplate.from_template("{country}의 수도는?") | ChatOpenAI() | output_parser

chain2 = PromptTemplate.from_template("{country}의 인구?") | ChatOpenAI() | output_parser

combined_chain = RunnableParallel(capital=chain1, populations=chain2)

result = combined_chain.invoke({ 'country': '대한민국' })
result

### RunnableParallel을 사용한 chain 병렬 실행 예시 #2
- 체인 1: 검색 가능한 URL을 생성
- 체인 2: 해당 주제에 대한 간단한 설명 생성

In [ ]:
url_prompt = PromptTemplate.from_template("'{topic}'에 대해 검색할 수 있는 네이버 검색 URL을 만들어줘.")
url_chain = url_prompt | ChatOpenAI() | output_parser

desc_prompt = PromptTemplate.from_template("'{topic}'에 대해 초등학생도 이해할 수 있게 설명해줘.")
desc_chain = desc_prompt | ChatOpenAI() | output_parser

combined_chain = RunnableParallel(url=url_chain, desc=desc_chain)
result = combined_chain.invoke({ 'topic': 'ESG' })
print(result)

※ 실습: 나만의 여행 정보 병렬 가이드 만들기

+ PromptTemplate 1 →
    - "{도시명}의 날씨는 어때?"
        - 예시: "제주의 날씨는 어때?"

+ PromptTemplate 2 →
    - "{도시명}에서 즐길 수 있는 대표 음식은 뭐야?"
        - 예시: "제주에서 즐길 수 있는 대표 음식은 뭐야?"

+ 두 PromptTemplate을 각각 LLM과 연결

+ RunnableParallel을 사용하여 두 개의 체인을 병렬 실행

+ {"city": "제주"}를 입력하여 두 응답을 동시에 출력할 것

In [ ]:
# YOUR CODE

## ChatPromptTemplate

ChatPromptTemplate 은 **대화목록을 프롬프트로 주입**하고자 할 때 활용할 수 있음.

메시지는 튜플(tuple) 형식으로 구성하며, (role, message) 로 구성하여 리스트로 생성.

- Role
    + "system": 시스템 설정 메시지. 주로 전역설정과 관련된 프롬프트입니다.  
    + "human" : 사용자 입력 메시지 입니다.
    + "ai": AI 의 답변 메시지입니다.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

In [ ]:
chat_prompt.format(country="대한민국")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# 챗 message 를 생성합니다.
messages = chat_template.format_messages(
    name="디랩", user_input="당신의 이름은 무엇입니까?"
)
messages

In [ ]:
llm.invoke(messages).content

이번에는 체인을 생성해보겠음.

In [ ]:
chain = chat_template | llm

chain.invoke({"name": "디랩", "user_input": "잘 지내십니까?"}).content

※ 실습: 나만의 AI 어시스턴트 대화 흐름 만들기

+ ChatPromptTemplate.from_messages()를 사용하여 아래의 대화 시나리오를 그대로 구현:

    역할	| 메시지

    system	| 당신은 {user_name}이라는 이름을 가진 공손한 AI 비서입니다.

    human	| 안녕하세요!

    ai	| 안녕하세요. 무엇을 도와드릴까요?

    human	| {question}




+ ChatOpenAI() 또는 ChatGPT LLM과 연결하여 응답 출력


+ invoke()를 사용하여 다음 입력으로 실행해볼 것:

    - {"user_name": "도우미봇", "question": "오늘 날씨 어때?"}

In [ ]:
# YOUR CODE